In [1]:
import pandas as pd
data=pd.read_csv('C:\\Users\\sh921\\Desktop\\YoutubeComment_preprocessed.csv')

data

,date,text,GameName
0,2015-10-13,갈수록 웃기고 퀄리티도 계속 높아지네 존잼,리그 오브 레전드
1,2015-10-13,얘가 솔직히 진짜 매력있는 캐릭턴데 고인,리그 오브 레전드
2,2015-10-13,역시 빛돌 빅 무리수 조쿠요 이런 쉰선한 바람이 에 필요하다,리그 오브 레전드
3,2015-10-13,갈수록 꿀잼 퀄도 높아지고,리그 오브 레전드
4,2015-10-13,역시 김정균 킨드레드템트리를 맞추네,리그 오브 레전드
...,...,...,...
962169,2015-12-03,몬길 몬스터 7성이랑 세나 6성이랑 싸우면 누가 이길까,넷마블 바둑nTV
962170,2015-12-03,신맵장난하나 달랑4개만들어놓고못함 모험하라는거야안하라는거야 초월캐릭터만들어서모험돌리...,넷마블 바둑nTV
962171,2015-12-03,근데알고보면 몬길이 왜망하는줄아나요 계속7성막주고 그러니깐 망하는거에요 게임하는사람...,넷마블 바둑nTV
962172,2015-12-03,와 세나모욕하네 몬스터길들이기 이런 세나모욕하지마라 세나가잘나가는이유 몬길 보다세...,넷마블 바둑nTV


In [2]:
from collections import Counter
#from tensorflow import keras
#from tensorflow.keras.preprocessing.text import Tokenizer

import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
from konlpy.tag import Okt
okt = Okt()
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models.wrappers.ldamallet import LdaMallet

In [3]:
import numpy as np
#데이터 프레임의 'text' 열의 값들을 str 형식으로 바꾸기
data.text = data.text.astype(str)


#text와 timestamp 열을 기준으로 중복된 데이터를 삭제, inplace : 데이터 프레임을 변경할지 선택(원본을 대체)
data.drop_duplicates(subset=['date','text','GameName'], inplace=True)

#한글이 아니면 빈 문자열로 바꾸기
data['text'] = data['text'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)

#빈 문자열 NAN 값으로 바꾸기
data = data.replace({'': np.nan})
data = data.replace(r'^\s*$', None, regex=True)

#NAN 이 있는 행은 삭제
data.dropna(how='any', inplace=True)

#인덱스 차곡차곡
data = data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(data.isnull().values.any()) 

False


In [4]:
data

,date,text,GameName
0,2015-10-13,갈수록 웃기고 퀄리티도 계속 높아지네 존잼,리그 오브 레전드
1,2015-10-13,얘가 솔직히 진짜 매력있는 캐릭턴데 고인,리그 오브 레전드
2,2015-10-13,역시 빛돌 빅 무리수 조쿠요 이런 쉰선한 바람이 에 필요하다,리그 오브 레전드
3,2015-10-13,갈수록 꿀잼 퀄도 높아지고,리그 오브 레전드
4,2015-10-13,역시 김정균 킨드레드템트리를 맞추네,리그 오브 레전드
...,...,...,...
955919,2015-12-03,몬길 몬스터 성이랑 세나 성이랑 싸우면 누가 이길까,넷마블 바둑nTV
955920,2015-12-03,신맵장난하나 달랑 개만들어놓고못함 모험하라는거야안하라는거야 초월캐릭터만들어서모험돌리...,넷마블 바둑nTV
955921,2015-12-03,근데알고보면 몬길이 왜망하는줄아나요 계속 성막주고 그러니깐 망하는거에요 게임하는사람...,넷마블 바둑nTV
955922,2015-12-03,와 세나모욕하네 몬스터길들이기 이런 세나모욕하지마라 세나가잘나가는이유 몬길 보다세...,넷마블 바둑nTV


In [ ]:
Data_list=data.text.values.tolist()
#(트윗 하나)씩 가져와서 명사만 추출한 후 리스트로 만들기
data_word=[]
for i in range(len(Data_list)):
    try:
        data_word.append(okt.nouns(Data_list[i]))
    except Exception as e:
        continue


#명사추출이 부자연스러운 경우 사전에 명사 추가

In [ ]:
data_word

In [ ]:
#토픽모델링에 필요한 Dictionary 와 Corpus 만들기 (no_below = 100 빈도수 100이하 단어 제거)
id2word=corpora.Dictionary(data_word)
id2word.filter_extremes(no_below = 100)
texts = data_word
corpus=[id2word.doc2bow(text) for text in texts]

In [ ]:
mallet_path = 'C:\\Users\\sh921\\Desktop\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet' 
import os
os.environ['MALLET_HOME'] = 'C:\\Users\\sh921\\Desktop\\mallet-2.0.8\\mallet-2.0.8\\'
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
print(ldamallet.show_topics(formatted=False))

In [ ]:
#lda모델의 일관성 점수 계산
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
#최적의 토픽 수를 찾기 위해 여러 토픽 수로 일관성을 계산하고 비교
def compute_coherence_values(dictionary, corpus, texts, limit, start=14, step=2):

    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=6, limit=41, step=2)

In [ ]:
# 일관성 점수 그래프 보기
limit=41; start=6; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# 토픽수 별 일관성 점수 확인
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# 토픽 수 선택하고 토픽들(단어) 확인
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=20))

In [ ]:
#트윗들의 토픽을 확인 (토픽별로)
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts):
    sent_topics_df = pd.DataFrame()

    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: 
                wp = ldamodel.show_topic(topic_num,topn=40)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=Data_list)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


df_dominant_topic=df_dominant_topic.sort_values(by=['Dominant_Topic'])
df_dominant_topic

In [ ]:
# 각 토픽에서 가장 대표적인 문장 찾기
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)


topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()
topic_counts.sort_index(inplace=True)

topic_contribution = round(topic_counts/topic_counts.sum(), 4)
topic_contribution

lda_inform = pd.concat([sent_topics_sorteddf_mallet, topic_counts, topic_contribution], axis=1)
lda_inform.columns=["Topic_Num", "Topic_Perc_Contrib", "Keywords", "Text", "Num_Documents", "Perc_Documents"]


lda_inform

In [ ]:
lda_inform.to_csv ("C:\\Users\\sh921\\Desktop\\LDA_youtube.csv", index = None)
lda_inform